# 🎨 Image Inpainting with Stable Diffusion 3.5 Medium

In this notebook, you'll launch a simple Gradio app that performs **image inpainting** using a powerful text-to-image model.  
You can **upload or draw on an image**, mark the area you want to modify using a **mask**, and describe what should appear there in natural language.

The app will use **Stable Diffusion 3.5 Medium** to fill in the masked region based on your prompt.

In [ ]:
import torch
from diffusers import StableDiffusionInpaintPipeline
from PIL import Image
import gradio as gr

# Load SD2 inpainting
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting",
    torch_dtype=torch.float16
).to("cuda")
pipe.enable_attention_slicing()

def inpaint(edited: dict, prompt: str):
    if not edited or not prompt.strip():
        return None

    bg: Image.Image = edited["background"]
    layers = edited["layers"]
    if bg is None or not layers:
        return None

    image = bg.convert("RGB").resize((512, 512))

    # Get the first layer’s alpha channel
    alpha = layers[0].getchannel("A").resize((512, 512))
    mask = alpha.point(lambda p: 255 if p > 0 else 0)  # white where painted

    image.show()
    mask.show()

    result = pipe(prompt=prompt, image=image, mask_image=mask).images[0]
    return result

gr.Interface(
    fn=inpaint,
    inputs=[
        gr.ImageEditor(label="Upload image & draw mask",
                       image_mode="RGBA", type="pil"),
        gr.Textbox(label="Prompt")
    ],
    outputs=gr.Image(label="Inpainted"),
    title="Stable Diffusion 2 Inpainting",
    description="Draw on the image, enter a prompt, and inpaint."
).launch(server_name="0.0.0.0", server_port=8080)